In [1]:
# import necessary library
import pandas as pd
import math
import seaborn as sns
from matplotlib import pyplot as plt
import re
import numpy as np
import geopandas as gpd
from shapely.geometry import Point # Point class
from shapely.geometry import shape

C:\Users\mmmiah\.conda\envs\geo_env\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
# surpres all warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# avoid unecessary zeros after interger year
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [14]:
# read eco_strava matched data
eco_stv=gpd.read_file('D:/Bike_ped Exposure Model Project/Processed Data/Eco_Strava_matching.shp')
eco_stv=eco_stv.drop(['year','OBJECTID','Join_Count','TARGET_FID'],axis=1)
eco_stv.head()

,ID,latitude,longitude,edgeUID,osmId,dist,geometry
0,3575,32.66,-117.07,305836426.00,234828855.00,d11,POINT (-117.07062 32.65733)
1,19209,32.66,-117.07,305836426.00,234828855.00,d11,POINT (-117.07162 32.65739)
2,59064,32.77,-117.02,306069075.00,436303561.00,d11,POINT (-117.01993 32.76747)
3,3578,32.58,-117.13,305674585.00,149560879.00,d11,POINT (-117.13077 32.58399)
4,3577,32.59,-117.11,305679029.00,6007743.00,d11,POINT (-117.10702 32.58770)


In [5]:
eco_stv.shape

(404, 11)

In [17]:
# read all eco data
eco=pd.read_csv('D:/Bike_ped Exposure Model Project/Processed Data/Eco_combined_data_before_QA_QC.csv')
eco16=eco.loc[eco['year']==2016]
eco16=eco16.drop_duplicates(subset=['ID','latitude','longitude'], keep='first')
eco16=eco16[["ID",'year','latitude','longitude']]

eco17=eco.loc[eco['year']==2017]
eco17=eco17.drop_duplicates(subset=['ID','latitude','longitude'], keep='first')
eco17=eco17[["ID",'year','latitude','longitude']]

eco18=eco.loc[eco['year']==2018]
eco18=eco18.drop_duplicates(subset=['ID','latitude','longitude'], keep='first')
eco18=eco18[["ID",'year','latitude','longitude']]

eco19=eco.loc[eco['year']==2019]
eco19=eco19.drop_duplicates(subset=['ID','latitude','longitude'], keep='first')
eco19=eco19[["ID",'year','latitude','longitude']]
eco19.head()

,ID,year,latitude,longitude
26304,671,2019,37.79,-122.46
61368,1127,2019,37.80,-122.42
96432,2415,2019,37.77,-122.44
131496,2691,2019,35.29,-120.66
166560,2692,2019,35.29,-120.66


In [23]:
eco19.shape,eco18.shape,eco17.shape,eco16.shape

((475, 4), (475, 4), (475, 4), (475, 4))

In [45]:
# add year with eco_strava 
eco_stv19=pd.merge(eco19,eco_stv, left_on=['ID','latitude','longitude'], right_on=['ID','latitude','longitude'], how='inner')
eco_stv18=pd.merge(eco18,eco_stv, left_on=['ID','latitude','longitude'], right_on=['ID','latitude','longitude'], how='inner')
eco_stv17=pd.merge(eco17,eco_stv, left_on=['ID','latitude','longitude'], right_on=['ID','latitude','longitude'], how='inner')
eco_stv17=eco_stv17.drop(['geometry'],axis=1)
eco_stv16=pd.merge(eco16,eco_stv, left_on=['ID','latitude','longitude'], right_on=['ID','latitude','longitude'], how='inner')
eco_stv16=eco_stv16.drop(['geometry'],axis=1)
eco_stv16.head()

,ID,year,latitude,longitude,edgeUID,osmId,dist
0,671,2016,37.79,-122.46,308723101.00,215845474.00,d4
1,1127,2016,37.80,-122.42,308779961.00,87297166.00,d4
2,2415,2016,37.77,-122.44,308725144.00,907910631.00,d4
3,2694,2016,33.68,-117.66,306405706.00,59238912.00,d12
4,2695,2016,33.67,-117.65,306408391.00,43826849.00,d12


In [46]:
eco_stv16.columns

Index(['ID', 'year', 'latitude', 'longitude', 'edgeUID', 'osmId', 'dist'], dtype='object')

In [22]:
eco_stv19.shape,eco_stv18.shape,eco_stv17.shape,eco_stv16.shape

((249, 8), (249, 8), (249, 8), (249, 8))

In [26]:
# read 2019 strava data
d1d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D1/59f453a1ea74ff48d88556f72ba70b289413e3c9988a49f55e3d722a06f0bd08-1651249687876.csv')
d2d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D2/20893741ac1090f60daf98c1c5fbda5781de61ec0cfac405ac696082a3c3913c-1651249705530.csv')
d3d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D3/dee63afb3a1adb1b662700f92d8c66eb58ee2dc702c98880c80ffe6b57983118-1651249721501.csv')
d4d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D4/7e3ac4a93435f0ee9bd5ca4d3c0695a89d83613a09ae96bc14b4bc9cdaf2dbfc-1651518670665.csv')
d5d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D5/ea61d5f36b0efe14dd85ab36756c8082f6f0bc38e7b2bd01d87da4cc9e997dce-1651518685821.csv')
d6d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D6/343c5fd09a7e61d6226038b77ba41d9d673a2a2bb78e0c4efa0b8d4dc9e0c7f1-1651518708125.csv')
d7d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D7/780268bacba82a74eb5a25991baa2656a5ee05bcb3830379d46b26cdf36bf1e5-1651606611912.csv')
d8d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D8/2bc3c7efd5db2e5823f3f9b3f8af13f56e588adf635dcf29da5506ea49bacee4-1651606628564.csv')
d9d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D9/752912d517b003fd103580e3d58726805be1ec9c264aff9401b41ba46be74c9e-1651606647084.csv')
d10d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D10/b33a496702ca56dfbc68aa7f90aef3c4a1187de5fc1d6cc6b89b0fd0955847f0-1651682235585.csv')
d11d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D11/9a0657f7164b7112f9a241c67e731e590b2a3c5b17216728b041e9476ae4ebea-1651682309481.csv')
d12d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2019/2019/2019/all_edges_yearly_2019_ride_D12/b5679e9cff99ad7f276d6c7e480190bdc1ddb518fa1327066a9ab73ce1a76e51-1651682326989.csv')
CA_Strava_data19=pd.concat([d1d,d2d,d3d,d4d,d5d,d6d,d7d,d8d,d9d,d10d,d11d,d12d],axis=0)
CA_Strava_data19.head()

,edge_uid,activity_type,year,forward_trip_count,reverse_trip_count,forward_people_count,reverse_people_count,forward_commute_trip_count,reverse_commute_trip_count,forward_leisure_trip_count,...,reverse_20_34_people_count,forward_35_54_people_count,reverse_35_54_people_count,forward_55_64_people_count,reverse_55_64_people_count,forward_65_plus_people_count,reverse_65_plus_people_count,forward_average_speed,reverse_average_speed,osm_reference_id
0,307087344,Ride,2019,0,5,0,5,0,0,0,...,0,0,0,0,0,0,0,0.00,1.75,10249524
1,307093461,Ride,2019,160,255,35,100,10,40,150,...,25,15,40,5,15,5,10,1.97,10.44,10266924
2,307093816,Ride,2019,125,115,15,10,95,85,30,...,0,5,10,0,0,0,0,5.39,7.15,396489943
3,307094834,Ride,2019,15,10,10,5,0,0,15,...,0,0,0,5,0,0,0,3.40,6.29,10249947
4,307095856,Ride,2019,75,70,35,15,0,5,75,...,5,15,5,10,5,5,0,4.55,3.62,136229365


In [35]:
eco_stv19_data=pd.merge(eco_stv19,CA_Strava_data19, left_on=['edgeUID','osmId','year'],right_on=['edge_uid','osm_reference_id','year'],how='inner')
eco_stv19_data.shape

(244, 41)

In [29]:
# read 2018 data
# Read distrcit 1-12 Strava data
d1d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D1/6ef13ecebbf2b7a8bd7bf242a08a1187e5e74ede8d1e2c1f40f21f9a71379e55-1650903378805.csv')
d2d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D2/44cf2e03b96404b9872bf550eaea24f813317d0d5a817cfcea9fad696805a734-1650903428425.csv')
d3d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D3/255a92b95d28169d37fc67c83e39e444e0643dfd3bfaf6fec1da60893e02a9e2-1650903448299.csv')
d4d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D4/075a1d3c2bcaa8641098036f74342333afbdaa711e1ae3b7a5688f1c4f0eaf71-1650987233391.csv')
d5d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D5/c866c5664e8223e5c800237e3565776dade2e0961603ae6c87310cbdbf8ba9af-1650987250914.csv')
d6d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D6/a3feac8689d6c00d604c7609d7c7d7a8c6ddb34f847874598f718c8e8022ac17-1650987275159.csv')
d7d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D7/ea612ab7e36f3e26b9be6c34fc9b78908045107a144f18e59653f45c3c5fafdf-1651076571753.csv')
d8d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D8/f308fe3ecdf647a71041f0d90778d3489ed937495a9a303476a9f9fa273773c5-1651076594138.csv')
d9d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D9/972916ce8ab7a5261a5dcc7dec4d61b5ba6b66173d4e67b5ec3e757deb6d07e4-1651076614829.csv')
d10d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D10/800cbbe4f28cdcdf8ccc1cd656e6d709811118ff55be4f92a5b8fb4cc27d6008-1651163735851.csv')
d11d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D11/292b5f0d8a1847f7076afa78c3d30319a742d0fdc1adbab22eea1b936cdba43c-1651163755994.csv')
d12d=pd.read_csv('D:/Bike_ped Exposure Model Project/Strava 2018/2018/2018/all_edges_yearly_2018_ride_D12/e92c8e0909e62306b6d616045980c526c280346cb41e1e6bc723f5bd63c66e44-1651163779808.csv')
CA_Strava_data18=pd.concat([d1d,d2d,d3d,d4d,d5d,d6d,d7d,d8d,d9d,d10d,d11d,d12d],axis=0)
CA_Strava_data18.head()

,edge_uid,activity_type,year,forward_trip_count,reverse_trip_count,forward_people_count,reverse_people_count,forward_commute_trip_count,reverse_commute_trip_count,forward_leisure_trip_count,...,reverse_20_34_people_count,forward_35_54_people_count,reverse_35_54_people_count,forward_55_64_people_count,reverse_55_64_people_count,forward_65_plus_people_count,reverse_65_plus_people_count,forward_average_speed,reverse_average_speed,osm_reference_id
0,307078653,Ride,2018,25,15,15,15,0,0,25,...,5,5,5,0,0,5,0,4.49,6.24,10306659
1,307080403,Ride,2018,10,5,10,5,0,0,10,...,0,5,0,0,0,0,0,3.43,3.35,238818704
2,307081105,Ride,2018,350,90,340,85,0,0,350,...,10,115,40,50,20,25,5,5.97,3.84,826580917
3,307098773,Ride,2018,5,10,5,10,0,0,5,...,5,5,5,0,0,0,0,8.15,6.14,34576425
4,307106597,Ride,2018,0,70,0,15,0,40,0,...,0,0,5,0,5,0,0,0.00,5.83,759146979


In [36]:
eco_stv18_data=pd.merge(eco_stv18,CA_Strava_data18, left_on=['edgeUID','osmId','year'],right_on=['edge_uid','osm_reference_id','year'],how='inner')
eco_stv18_data.shape

(246, 41)

In [37]:
eco_stv1819=pd.concat([eco_stv18_data,eco_stv19_data],axis=0)
eco_stv1819.shape

(490, 41)

In [38]:
eco_stv1819['stv_total']=eco_stv1819['forward_trip_count']+eco_stv1819['reverse_trip_count']
eco_stv1819['stv_com_total']=eco_stv1819['forward_commute_trip_count']+eco_stv1819['reverse_commute_trip_count']
eco_stv1819['stv_rec_total']=eco_stv1819['forward_leisure_trip_count']+eco_stv1819['reverse_leisure_trip_count']
eco_stv1819['stv_com_ratio']=eco_stv1819['stv_com_total']/eco_stv1819['stv_total']
eco_stv1819['stv_rec_ratio']=eco_stv1819['stv_rec_total']/eco_stv1819['stv_total']
eco_stv1819.head()

,ID,year,latitude,longitude,edgeUID,osmId,dist,geometry,edge_uid,activity_type,...,forward_65_plus_people_count,reverse_65_plus_people_count,forward_average_speed,reverse_average_speed,osm_reference_id,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio
0,671,2018,37.79,-122.46,308723101.00,215845474.00,d4,POINT (-122.45944 37.79116),308723101,Ride,...,15,20,3.28,6.47,215845474,4890,260,4630,0.05,0.95
1,1127,2018,37.80,-122.42,308779961.00,87297166.00,d4,POINT (-122.41949 37.79617),308779961,Ride,...,5,0,5.60,5.20,87297166,405,150,255,0.37,0.63
2,44181,2018,37.80,-122.42,308779961.00,87297166.00,d4,POINT (-122.41949 37.79617),308779961,Ride,...,5,0,5.60,5.20,87297166,405,150,255,0.37,0.63
3,2415,2018,37.77,-122.44,308725144.00,907910631.00,d4,POINT (-122.43629 37.77420),308725144,Ride,...,0,85,2.84,5.17,907910631,30655,11705,18950,0.38,0.62
4,44161,2018,37.77,-122.44,308725144.00,907910631.00,d4,POINT (-122.43629 37.77420),308725144,Ride,...,0,85,2.84,5.17,907910631,30655,11705,18950,0.38,0.62


In [39]:
eco_stv1819.columns

Index(['ID', 'year', 'latitude', 'longitude', 'edgeUID', 'osmId', 'dist',
       'geometry', 'edge_uid', 'activity_type', 'forward_trip_count',
       'reverse_trip_count', 'forward_people_count', 'reverse_people_count',
       'forward_commute_trip_count', 'reverse_commute_trip_count',
       'forward_leisure_trip_count', 'reverse_leisure_trip_count',
       'forward_morning_trip_count', 'reverse_morning_trip_count',
       'forward_evening_trip_count', 'reverse_evening_trip_count',
       'forward_male_people_count', 'reverse_male_people_count',
       'forward_female_people_count', 'reverse_female_people_count',
       'forward_unspecified_people_count', 'reverse_unspecified_people_count',
       'forward_13_19_people_count', 'reverse_13_19_people_count',
       'forward_20_34_people_count', 'reverse_20_34_people_count',
       'forward_35_54_people_count', 'reverse_35_54_people_count',
       'forward_55_64_people_count', 'reverse_55_64_people_count',
       'forward_65_plus_people

In [47]:
eco_stv1819v1=eco_stv1819[['ID', 'year', 'latitude', 'longitude', 'edgeUID', 'osmId', 'dist',
       'stv_total', 'stv_com_total', 'stv_rec_total', 'stv_com_ratio',
       'stv_rec_ratio']]

In [48]:
eco_stv_all=pd.concat([eco_stv1819v1,eco_stv17,eco_stv16],axis=0)
eco_stv_all.shape

(988, 12)

In [49]:
eco_stv_all.to_csv('D:/Bike_ped Exposure Model Project/Processed Data/eco_stv_all.csv')